In [2]:
import pandas as pd
import numpy as np

import datetime as dt

import holidays
from calendra.asia import Qatar
from calendra.asia import HongKong

In [3]:
calls_import = pd.read_csv('../data/Calls_Table_data.csv', delimiter='\t', encoding='utf-16')

In [120]:
import pandas as pd
import numpy as np
import datetime as dt

class DataRetrieval():
    
    #def __init__(self):

    
    def get_calls_data(self, filepath='../data/Calls_Table_data.csv', delimiter='\t'):
        """Retrieves call data from filepath"""
        df = pd.read_csv(filepath, delimiter='\t', encoding='utf-16')
        df.drop_duplicates(inplace=True)
        df.reset_index(drop=True, inplace=True)        
        return df
    
    def get_weather_data(self, filepath='../data/historical_weather.csv'):
        """Retrieves weather data from filepath"""
        df = pd.read_csv(filepath)
        df['date'] = pd.to_datetime(df['DATE']).dt.date
        weather_hist = (df[['date', 'TMAX', 'PRCP', 'SNOW']]
                        .rename(columns={'PRCP':'precip', 'TMAX':'temp_max',
                                         'PRCP':'precip', 'SNOW':'snow'}))
        weather_hist['precip^2'] = weather_hist['precip']**2
        weather_hist['snow^2'] = weather_hist['snow']**2      
        return weather_hist
    
    def get_seahawks_schedule(self, filepath='../data/seahawks_schedule.csv'):
        """
        Retrieves weather data from filepath
        
        Will need to rewrite scraper used to retrieve this data
        """
        df_in = pd.read_csv(filepath)
        df_dropna = df_in.copy()[df_in['Opp'].notna()].reset_index(drop=True)
        df = df_dropna.copy()[df_dropna['Opp'] != 'Bye Week'].reset_index(drop=True)
        
        
        df['date'] = ''
        for i in range(len(df)):
            df['date'][i] = (f"{df.iloc[i]['Date']}, {df.iloc[i]['year']}")
            
        df['date'] = pd.to_datetime(df['date']).dt.date
        
        df['seahawks_game'] = ''
        for i in range(len(df)):
            if type(df.iloc[i]['Unnamed: 10']) == str:
                if df.iloc[i]['Week'] == 'SuperBowl':
                    df['seahawks_game'][i] = 'SuperBowl'
                elif df.iloc[i]['Week'] in ['Wild Card', 'Division','Conf. Champ.']:
                    df['seahawks_game'][i] = 'away_Playoffs'
                else:
                    df['seahawks_game'][i] = 'away_Regular'
            else:     
                if df.iloc[i]['Week'] == 'SuperBowl':
                    df['seahawks_game'][i] = 'SuperBowl'
                elif df.iloc[i]['Week'] in ['Wild Card', 'Division','Conf. Champ.']:
                    df['seahawks_game'][i] = 'home_Playoffs'
                else:
                    df['seahawks_game'][i] = 'home_Regular'
        return df[['date', 'seahawks_game']]
    
        
class FeaturizeCalls():
    """Clean incoming df to fit into model"""
    
    #def __init__(self):
    
    def fit(self, dataframe, y=None):
        self.dataframe = dataframe
        return self

    
    def transform(self):
        """tranform and clean incoming training or test"""
        df = self.dataframe.copy()      

        df['dt_time'] = pd.to_datetime(df['ORIG_TIME_QUEUED'])
        df['date'] = df['dt_time'].dt.date
        df['year'] = df['dt_time'].dt.year
        df['month'] = df['dt_time'].dt.month
        df['day'] = df['dt_time'].dt.day
        df['day_of_week'] = df['dt_time'].dt.weekday
        df['month_day'] = df['dt_time'].dt.strftime('%m/%d')
        df['month_weekday'] = df['dt_time'].dt.strftime('%b_%a')
        df['month'] = df['dt_time'].dt.strftime('%m/%d')        
        return df
    
    def make_holiday_dummies(self, holiday_dict):
        _holidays = []
        for date in sorted(holiday_dict.keys()):
            for name in holiday_dict[date]:
                _holidays.append([date, name])
        _holidays = pd.DataFrame(_holidays, columns=['date', 'holiday'])
        return pd.get_dummies(_holidays.set_index('date')).reset_index()

    def make_event_dummies(self, event_dict=None):
        if event_dict == None:
            
            events_dict = ({
                'Pride Parade' : ['6/30/2019', '6/24/2018', '6/25/2017', '6/26/2016', '6/28/2015',
                                  '6/29/2014', '6/30/2013', '6/24/2012', '6/26/2011', '6/27/2010'],
                'Seafair' : ['8/2/2019', '8/3/2019', '8/4/2019', '8/3/2018', '8/4/2018', '8/5/2018',
                             '8/4/2017', '8/5/2017', '8/6/2017', '8/5/2016', '8/6/2016', '8/7/2016',
                             '7/31/2015', '8/1/2015', '8/2/2015', '8/1/2014', '8/2/2014', '8/3/2014',
                             '8/2/2013', '8/3/2013', '8/4/2013', '8/3/2012', '8/4/2012', '8/5/2012',
                             '8/5/2011', '8/6/2011', '8/7/2011', '8/6/2010', '8/7/2010', '8/8/2010' ],
                'Soltice Parade': ['6/30/2019', '6/16/2018', '6/17/2017', '6/18/2016', '6/20/2015',
                                   '6/21/2014', '6/22/2013', '6/16/2012', '6/18/2011', '6/19/2010'],
                'Womens March' : ['1/19/2019', '1/20/2018', '1/21/2017'],})

        _events = []
        for event in events_dict.keys():
            for day in events_dict[event]:
                _events.append([dt.datetime.strptime(day, '%m/%d/%Y'), event])
                
        _events = pd.DataFrame(_events, columns=['date','local_event'])
        _events['date'] = _events['date'].dt.date
        return pd.get_dummies(_events.set_index('date')).reset_index()
        
 
    
    def join_data(self, df, new_data):
        return df.join(new_data.set_index('date'), on='date')

In [ ]:
for event in events.keys():
    for day in events[event]:
        seattle_events.append([dt.datetime.strptime(day, '%m/%d/%Y'), event])
seattle_events = pd.DataFrame(seattle_events, columns=['date','local_event'])
seattle_events['date'] = seattle_events['date'].dt.date

In [23]:
import pandas as pd
import numpy as np
import datetime as dt

import holidays
from calendra.asia import Qatar
from calendra.asia import HongKong

class SeattleHolidays:
    
    
    class CustomHolidays(holidays.US):
        def _populate(self, year=2019, start_year=2009, end_year=2030):
            # Populate the holiday list with the default US holidays
            holidays.US._populate(self, year)
            # Example: Add Ninja Turtle Day
            #self[dt.date(year, 7, 13)] = "Ninja Turtle Day"
            for year in range(start_year, end_year):
                # Add Valentine's day
                self[dt.date(year, 2, 14)] = "Valentines Day"
                # Add St Patricks Day
                self[dt.date(year, 3, 17)] = "St Patricks Day"
                # Add Easter
                self[holidays.easter(year=year)] = "Easter"
                # Add Good Friday
                self[holidays.easter(year=year)  -  dt.timedelta(days=2)] = "Good Friday"
                # Add Christmas Eve
                self[dt.date(year, 12, 24)] = "Christmas Eve"
                # Add New Years Eve
                self[dt.date(year, 12, 31)] = "New Years Eve"
                # Add Chinese New Year
                chinese = HongKong()
                for date, label in chinese.get_chinese_new_year(year):
                    self[date] = label


    class IslamicHolidays(holidays.HolidayBase):
        qatar_holidays = Qatar()
        def _populate(self, year=2019, start_year=2009, end_year=2030):
            # Populate the holiday list with blank base holidays
            holidays.HolidayBase._populate(self, year)
            for year in range(start_year, end_year):
                days = qatar_holidays.get_calendar_holidays(year)
                # Add Ramadan
                for i in range(1, len(days)):
                    if qatar_holidays.get_calendar_holidays(year)[i][1] == 'Start of ramadan':
                        for day in range(30):
                            self[qatar_holidays.get_calendar_holidays(year)[i][0] 
                                 + dt.timedelta(days=day-1)] = "Ramadan"
                    else:
                        self[qatar_holidays.get_calendar_holidays(2018)[1][0] 
                             - dt.timedelta(days=2)] = qatar_holidays.get_calendar_holidays(year)[i][1]


    class JewishHolidays(holidays.HolidayBase):
        def retrieve_data(self, filepath):
            df = pd.read_csv(filepath)
            return df

        def get_holidays(self, paths_list):
            df = self.retrieve_data(paths_list[0])
            for filepath in paths_list[1 : ]:
                cal = self.retrieve_data(filepath)
                df = pd.concat([df, cal])
            return df

        def _populate(self, year=2019, paths_list=['../data/hebcal_2010_usa.csv',
                                                   '../data/hebcal_2015_usa.csv', 
                                                   '../data/hebcal_2020_usa.csv',
                                                   '../data/hebcal_2025_usa.csv'],
                      start_year=2009, end_year=2030):
            hebcal = self.get_holidays(paths_list)
            hebcal.reset_index(drop=True, inplace=True)
            hebcal['date'] = pd.to_datetime(hebcal["Start Date"]).dt.date
            # Populate the holiday list with blank base holidays
            holidays.HolidayBase._populate(self, year)
            for year in range(start_year, end_year):
                for i in range(len(hebcal)):
                    self[hebcal['date'][i]] = hebcal['Subject'][i]


In [121]:
retriever = DataRetrieval()

In [122]:
df = retriever.get_calls_data()

In [123]:
weather_df = retriever.get_weather_data()

In [124]:
seahawks_schedule = retriever.get_seahawks_schedule()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [125]:
seahawks_schedule

,date,seahawks_game
0,2010-09-12,home_Regular
1,2010-09-19,away_Regular
2,2010-09-26,home_Regular
3,2010-10-03,away_Regular
4,2010-10-17,away_Regular
5,2010-10-24,home_Regular
6,2010-10-31,away_Regular
7,2010-11-07,home_Regular
8,2010-11-14,away_Regular
9,2010-11-21,away_Regular


In [ ]:
seattle_dv_eda.ipynb

In [82]:
cf = FeaturizeCalls()

In [83]:
cf.fit(df)

In [84]:
calls_df = cf.transform()

In [85]:
cf.join_data(calls_df, weather_df)

,BEAT,CALL_CODE,CALL_DESC,CALL GROUP,YEAR,MAP COLOR,CALL VOLUME,Current Year Mark,Select View,Boundary Selection,...,month,day,day_of_week,month_day,month_weekday,temp_max,precip,snow,precip^2,snow^2
0,R3,082,DV - DOMESTIC VIOLENCE (ARREST DISCRETIONARY),DOMESTIC DISTURBANCE/VIOLENCE,2010,S,1,Previous Year,Monthly View,COLUMBIA CITY,...,01/08,8,4,01/08,Jan_Fri,46,0.81,0.0,0.6561,0.0
1,N2,081,"DV - ARGUMENTS, DISTURBANCE (NO ARREST)",DOMESTIC DISTURBANCE/VIOLENCE,2010,N,1,Previous Year,Monthly View,BITTERLAKE,...,01/08,8,4,01/08,Jan_Fri,46,0.81,0.0,0.6561,0.0
2,N3,081,"DV - ARGUMENTS, DISTURBANCE (NO ARREST)",DOMESTIC DISTURBANCE/VIOLENCE,2010,N,1,Previous Year,Monthly View,NORTHGATE,...,01/08,8,4,01/08,Jan_Fri,46,0.81,0.0,0.6561,0.0
3,F2,081,"DV - ARGUMENTS, DISTURBANCE (NO ARREST)",DOMESTIC DISTURBANCE/VIOLENCE,2010,SW,1,Previous Year,Monthly View,SOUTH DELRIDGE,...,01/08,8,4,01/08,Jan_Fri,46,0.81,0.0,0.6561,0.0
4,Q3,080,DV - DOMESTIC THREATS BY PHONE OR WRITING,DOMESTIC DISTURBANCE/VIOLENCE,2010,W,1,Previous Year,Monthly View,SLU/CASCADE,...,01/20,20,2,01/20,Jan_Wed,57,0.00,0.0,0.0000,0.0
5,W1,087,DV - ENFORCE COURT ORDER (ARREST MANDATED),DOMESTIC DISTURBANCE/VIOLENCE,2010,SW,1,Previous Year,Monthly View,ALKI,...,01/04,4,0,01/04,Jan_Mon,49,0.98,0.0,0.9604,0.0
6,B3,085,DV - SERVICE OF COURT ORDER,DOMESTIC DISTURBANCE/VIOLENCE,2010,N,1,Previous Year,Monthly View,FREMONT,...,01/04,4,0,01/04,Jan_Mon,49,0.98,0.0,0.9604,0.0
7,R2,081,"DV - ARGUMENTS, DISTURBANCE (NO ARREST)",DOMESTIC DISTURBANCE/VIOLENCE,2010,S,1,Previous Year,Monthly View,CLAREMONT/RAINIER VISTA,...,01/04,4,0,01/04,Jan_Mon,49,0.98,0.0,0.9604,0.0
8,S3,081,"DV - ARGUMENTS, DISTURBANCE (NO ARREST)",DOMESTIC DISTURBANCE/VIOLENCE,2010,S,1,Previous Year,Monthly View,RAINIER BEACH,...,01/20,20,2,01/20,Jan_Wed,57,0.00,0.0,0.0000,0.0
9,S1,081,"DV - ARGUMENTS, DISTURBANCE (NO ARREST)",DOMESTIC DISTURBANCE/VIOLENCE,2010,S,1,Previous Year,Monthly View,MID BEACON HILL,...,01/20,20,2,01/20,Jan_Wed,57,0.00,0.0,0.0000,0.0


In [68]:
holiday_test = SeattleHolidays.CustomHolidays()

In [69]:
holiday_test._populate()

In [70]:
event_dummies = cf.make_event_dummies()

In [72]:
cf.add_dummies(new_df, event_dummies)

,BEAT,CALL_CODE,CALL_DESC,CALL GROUP,YEAR,MAP COLOR,CALL VOLUME,Current Year Mark,Select View,Boundary Selection,...,year,month,day,day_of_week,month_day,month_weekday,local_event_Pride Parade,local_event_Seafair,local_event_Soltice Parade,local_event_Womens March
0,R3,082,DV - DOMESTIC VIOLENCE (ARREST DISCRETIONARY),DOMESTIC DISTURBANCE/VIOLENCE,2010,S,1,Previous Year,Monthly View,COLUMBIA CITY,...,2010,01/08,8,4,01/08,Jan_Fri,NaN,NaN,NaN,NaN
1,N2,081,"DV - ARGUMENTS, DISTURBANCE (NO ARREST)",DOMESTIC DISTURBANCE/VIOLENCE,2010,N,1,Previous Year,Monthly View,BITTERLAKE,...,2010,01/08,8,4,01/08,Jan_Fri,NaN,NaN,NaN,NaN
2,N3,081,"DV - ARGUMENTS, DISTURBANCE (NO ARREST)",DOMESTIC DISTURBANCE/VIOLENCE,2010,N,1,Previous Year,Monthly View,NORTHGATE,...,2010,01/08,8,4,01/08,Jan_Fri,NaN,NaN,NaN,NaN
3,F2,081,"DV - ARGUMENTS, DISTURBANCE (NO ARREST)",DOMESTIC DISTURBANCE/VIOLENCE,2010,SW,1,Previous Year,Monthly View,SOUTH DELRIDGE,...,2010,01/08,8,4,01/08,Jan_Fri,NaN,NaN,NaN,NaN
4,Q3,080,DV - DOMESTIC THREATS BY PHONE OR WRITING,DOMESTIC DISTURBANCE/VIOLENCE,2010,W,1,Previous Year,Monthly View,SLU/CASCADE,...,2010,01/20,20,2,01/20,Jan_Wed,NaN,NaN,NaN,NaN
5,W1,087,DV - ENFORCE COURT ORDER (ARREST MANDATED),DOMESTIC DISTURBANCE/VIOLENCE,2010,SW,1,Previous Year,Monthly View,ALKI,...,2010,01/04,4,0,01/04,Jan_Mon,NaN,NaN,NaN,NaN
6,B3,085,DV - SERVICE OF COURT ORDER,DOMESTIC DISTURBANCE/VIOLENCE,2010,N,1,Previous Year,Monthly View,FREMONT,...,2010,01/04,4,0,01/04,Jan_Mon,NaN,NaN,NaN,NaN
7,R2,081,"DV - ARGUMENTS, DISTURBANCE (NO ARREST)",DOMESTIC DISTURBANCE/VIOLENCE,2010,S,1,Previous Year,Monthly View,CLAREMONT/RAINIER VISTA,...,2010,01/04,4,0,01/04,Jan_Mon,NaN,NaN,NaN,NaN
8,S3,081,"DV - ARGUMENTS, DISTURBANCE (NO ARREST)",DOMESTIC DISTURBANCE/VIOLENCE,2010,S,1,Previous Year,Monthly View,RAINIER BEACH,...,2010,01/20,20,2,01/20,Jan_Wed,NaN,NaN,NaN,NaN
9,S1,081,"DV - ARGUMENTS, DISTURBANCE (NO ARREST)",DOMESTIC DISTURBANCE/VIOLENCE,2010,S,1,Previous Year,Monthly View,MID BEACON HILL,...,2010,01/20,20,2,01/20,Jan_Wed,NaN,NaN,NaN,NaN


In [ ]:
"""calls_full_df = calls_import

df.drop_duplicates(inplace=True)

df.reset_index(inplace=True)

df.columns

df['dt_time'] = pd.to_datetime(calls_full_df['ORIG_TIME_QUEUED'])

df['date'] = calls_full_df['dt_time'].dt.date

df['year'] = calls_full_df['dt_time'].dt.year

df['month'] = calls_full_df['dt_time'].dt.month

df['day'] = calls_full_df['dt_time'].dt.day

df['day_of_week'] = calls_full_df['dt_time'].dt.weekday

df['month_day'] = calls_full_df['dt_time'].dt.strftime('%m/%d')
    

df['month_weekday'] = calls_full_df['dt_time'].dt.strftime('%b_%a')

df['month'] = calls_full_df['dt_time'].dt.strftime('%m/%d')"""